In [1]:
!pip install dm-acme
!pip install dm-acme[reverb]
!pip install dm-acme[tf]
!pip install gym

You should consider upgrading via the '/home/pjotr/projects/poc/tutorials/datascience/venv/bin/python3.8 -m pip install --upgrade pip' command.
zsh:1: no matches found: dm-acme[reverb]
zsh:1: no matches found: dm-acme[tf]
You should consider upgrading via the '/home/pjotr/projects/poc/tutorials/datascience/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
environment_library = 'gym'

In [15]:
import IPython

from acme import environment_loop
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.agents.tf import ddpg
from acme.tf import networks
from acme.tf import utils as tf2_utils
from acme.utils import loggers
import numpy as np
import sonnet as snt
import gym
import time


environment = gym.make('MountainCarContinuous-v0')
environment = wrappers.GymWrapper(environment)  # To dm_env interface.

# Make sure the environment outputs single-precision floats.
environment = wrappers.SinglePrecisionWrapper(environment)

# Grab the spec of the environment.
environment_spec = specs.make_environment_spec(environment)

#@title Build agent networks

# Get total number of action dimensions from action spec.
num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)

# Create the shared observation network; here simply a state-less operation.
observation_network = tf2_utils.batch_concat

# Create the deterministic policy network.
policy_network = snt.Sequential([
    networks.LayerNormMLP((256, 256, 256), activate_final=True),
    networks.NearZeroInitializedLinear(num_dimensions),
    networks.TanhToSpec(environment_spec.actions),
])

# Create the distributional critic network.
critic_network = snt.Sequential([
    # The multiplexer concatenates the observations/actions.
    networks.CriticMultiplexer(),
#     networks.LayerNormMLP((512, 512, 256), activate_final=True),
    networks.LayerNormMLP((512, 512, 1), activate_final=True),
#     networks.DiscreteValuedHead(vmin=-150., vmax=150., num_atoms=51),
])

# Create a logger for the agent and environment loop.
agent_logger = loggers.TerminalLogger(label='agent', time_delta=10.)
env_loop_logger = loggers.TerminalLogger(label='env_loop', time_delta=10.)

# Create the D4PG agent.
agent = ddpg.DDPG(
    environment_spec=environment_spec,
    policy_network=policy_network,
    critic_network=critic_network,
    observation_network=observation_network,
    sigma=1.0,
    logger=agent_logger,
    checkpoint=False
)

# Create an loop connecting this agent to the environment created above.
env_loop = environment_loop.EnvironmentLoop(
    environment, agent, logger=env_loop_logger)

In [16]:
env_loop.run(num_episodes=10)

[Agent] Critic Loss = 0.038 | Policy Loss = 0.000 | Steps = 1 | Walltime = 0
[Env Loop] Episode Length = 999 | Episode Return = -51.893978118896484 | Episodes = 2 | Steps = 1998 | Steps Per Second = 60.058
[Agent] Critic Loss = 0.025 | Policy Loss = 0.000 | Steps = 432 | Walltime = 10.009
[Env Loop] Episode Length = 999 | Episode Return = -52.25935745239258 | Episodes = 6 | Steps = 5994 | Steps Per Second = 341.244
[Agent] Critic Loss = 0.030 | Policy Loss = 0.000 | Steps = 841 | Walltime = 20.015
[Env Loop] Episode Length = 999 | Episode Return = -51.88341522216797 | Episodes = 10 | Steps = 9990 | Steps Per Second = 333.790


In [17]:
timestep = environment.reset()
action = agent.select_action(timestep.observation)

while not timestep.last():
    environment.render()
    environment.step(action)
    time.sleep(0.0125)


KeyboardInterrupt: 